In [57]:
import pickle
import os
from od.social import SocialGroup
from od.network.types import ResourceAllocatorType,BaseStationType
from od.misc.interest import InterestConfig
from od.config import BS_PRESET,SUMO_SIM_SECONDS

In [126]:
statistics  = [None for _ in ResourceAllocatorType]
for _res_alloc, res_alloc in enumerate(ResourceAllocatorType):
    interest_config = InterestConfig(res_alloc,True,7,4419883)
    print(interest_config)
    with open("data/{}/{}/object.pickle".format(interest_config.rng_seed,str(interest_config)), "rb") as file:
                       statistics[_res_alloc] = pickle.load(file)

res_alloc_type(OMA) req_rsu(True) appdata_poisson(7)
res_alloc_type(NOMA_OPT) req_rsu(True) appdata_poisson(7)


In [120]:
class GV:
    NET_STATION_CONTROLLER = [
            bs_config for bs_config in BS_PRESET.values()
             if ((bs_config["type"] == BaseStationType.UMA)  or 
            (interest_config.req_rsu and bs_config["type"] == BaseStationType.UMI))
    ]
    class STATISTIC:
        def Log(data):
            print(data)

In [107]:
GV.NET_STATION_CONTROLLER

[{'color': (0, 0, 0, 255),
  'pos': (285, 245),
  'img': 'C:\\Users\\OD\\Desktop\\ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMA: 1>},
 {'color': (0, 0, 0, 255),
  'pos': (127, 217),
  'img': 'C:\\Users\\OD\\Desktop\\ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (52, 145),
  'img': 'C:\\Users\\OD\\Desktop\\ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (315, 395),
  'img': 'C:\\Users\\OD\\Desktop\\ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (270, 50),
  'img': 'C:\\Users\\OD\\Desktop\\ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  'type': <BaseStationType.UMI: 0>},
 {'color': (0, 0, 0, 255),
  'pos': (525, 95),
  'img': 'C:\\Users\\OD\\Desktop\\ProjectSumo/wifi.png',
  'width': 5,
  'height': 3.9,
  

In [129]:
for sg in SocialGroup:
    for header in statistics[0].sg_header[sg].keys():
        if(header not in statistics[1].sg_header[sg]):
            print("{} in statistic[0] but not in statistic[1]".format(header))
        else:
            s0 = statistics[0].sg_header[sg][header].time_bs_serv
            s1 = statistics[1].sg_header[sg][header].time_bs_serv
            for bs_serial,bs_config in enumerate(GV.NET_STATION_CONTROLLER):
                if(s0[bs_serial] != s1[bs_serial]):
                    print(
                        "{}/{}/{}: {}={},{}={}".format(
                            sg,
                            header,
                            bs_serial,
                            ResourceAllocatorType(0).name,
                            s0[bs_serial],
                            ResourceAllocatorType(1).name,
                            s1[bs_serial]
                        )
                    )

CRITICAL/1.126-2/0: OMA=252.3,NOMA_OPT=252.4
CRITICAL/0.140-2/0: OMA=252.3,NOMA_OPT=252.4
CRITICAL/1.135-2/0: OMA=252.5,NOMA_OPT=252.4
CRITICAL/1.153-1/0: OMA=252.5,NOMA_OPT=252.4
CRITICAL/1.85-0/0: OMA=252.5,NOMA_OPT=252.60000000000002
CRITICAL/1.85-1/0: OMA=252.5,NOMA_OPT=252.60000000000002
CRITICAL/1.153-3/0: OMA=252.5,NOMA_OPT=252.60000000000002
CRITICAL/1.162-8/0: OMA=253.3,NOMA_OPT=253.4
CRITICAL/1.162-9/0: OMA=253.3,NOMA_OPT=253.4
CRITICAL/2.26-3/0: OMA=253.3,NOMA_OPT=253.4
CRITICAL/2.26-3/1: OMA=253.4,NOMA_OPT=253.5
CRITICAL/2.26-3/2: OMA=253.4,NOMA_OPT=253.5
CRITICAL/2.26-3/3: OMA=253.4,NOMA_OPT=253.5
CRITICAL/2.26-3/5: OMA=253.4,NOMA_OPT=253.5
CRITICAL/2.28-5/0: OMA=253.4,NOMA_OPT=253.5
CRITICAL/1.126-9/0: OMA=253.4,NOMA_OPT=253.5
CRITICAL/1.126-10/0: OMA=253.4,NOMA_OPT=253.5
CRITICAL/0.140-13/0: OMA=253.4,NOMA_OPT=253.5
CRITICAL/0.149-9/0: OMA=253.4,NOMA_OPT=253.5
CRITICAL/0.149-10/0: OMA=253.4,NOMA_OPT=253.5
CRITICAL/0.149-11/0: OMA=253.4,NOMA_OPT=253.5
CRITICAL/0.99-10/0: 

CRITICAL/0.176-213/1: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-213/2: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-213/3: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-213/4: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-214/1: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-214/2: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-214/3: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-214/4: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-215/1: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-215/2: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-215/3: OMA=297.3,NOMA_OPT=297.5
CRITICAL/0.176-215/4: OMA=297.3,NOMA_OPT=297.5
CRITICAL/1.198-0/1: OMA=297.40000000000003,NOMA_OPT=297.6
CRITICAL/1.198-0/2: OMA=297.40000000000003,NOMA_OPT=297.6
CRITICAL/1.198-0/3: OMA=297.40000000000003,NOMA_OPT=297.6
CRITICAL/1.198-0/4: OMA=297.40000000000003,NOMA_OPT=297.6
CRITICAL/2.26-252/0: OMA=297.40000000000003,NOMA_OPT=297.3
CRITICAL/0.140-294/0: OMA=297.40000000000003,NOMA_OPT=297.3
CRITICAL/0.167-324/0: OMA=297.40000000000003,NOMA_OPT=297.3
CRITICAL/0.167-325/0: OMA

GENERAL/1.162-63/0: OMA=266.0,NOMA_OPT=266.1
GENERAL/1.162-63/1: OMA=285.7,NOMA_OPT=286.90000000000003
GENERAL/2.26-67/0: OMA=266.0,NOMA_OPT=266.2
GENERAL/2.26-67/1: OMA=285.7,NOMA_OPT=287.0
GENERAL/1.85-90/0: OMA=266.0,NOMA_OPT=266.1
GENERAL/1.85-90/1: OMA=285.7,NOMA_OPT=286.90000000000003
GENERAL/1.85-91/0: OMA=266.0,NOMA_OPT=266.1
GENERAL/1.85-91/1: OMA=285.7,NOMA_OPT=286.90000000000003
GENERAL/1.85-92/0: OMA=266.0,NOMA_OPT=266.1
GENERAL/1.85-92/1: OMA=285.7,NOMA_OPT=286.90000000000003
GENERAL/1.85-93/0: OMA=266.0,NOMA_OPT=266.1
GENERAL/1.85-93/1: OMA=285.7,NOMA_OPT=286.90000000000003
GENERAL/1.85-94/0: OMA=266.0,NOMA_OPT=266.2
GENERAL/1.85-94/1: OMA=285.8,NOMA_OPT=287.0
GENERAL/1.85-95/0: OMA=266.0,NOMA_OPT=266.2
GENERAL/1.85-95/1: OMA=285.8,NOMA_OPT=287.0
GENERAL/0.176-1/0: OMA=266.0,NOMA_OPT=266.2
GENERAL/0.176-1/1: OMA=285.8,NOMA_OPT=287.0
GENERAL/0.176-2/0: OMA=266.1,NOMA_OPT=266.2
GENERAL/0.176-2/1: OMA=285.8,NOMA_OPT=287.0
GENERAL/0.176-3/0: OMA=266.1,NOMA_OPT=266.2
GENERAL/0

GENERAL/2.26-174/0: OMA=285.0,NOMA_OPT=283.5
GENERAL/2.26-174/2: OMA=302.6,NOMA_OPT=307.5
GENERAL/2.26-174/3: OMA=290.3,NOMA_OPT=292.5
GENERAL/2.26-175/0: OMA=285.0,NOMA_OPT=283.5
GENERAL/2.26-175/2: OMA=302.6,NOMA_OPT=307.5
GENERAL/2.26-175/3: OMA=290.3,NOMA_OPT=292.5
GENERAL/0.176-98/0: OMA=285.0,NOMA_OPT=283.5
GENERAL/0.176-98/2: OMA=302.6,NOMA_OPT=307.5
GENERAL/0.176-98/3: OMA=290.3,NOMA_OPT=292.6
GENERAL/0.176-99/0: OMA=285.0,NOMA_OPT=283.5
GENERAL/0.176-99/2: OMA=302.6,NOMA_OPT=307.5
GENERAL/0.176-99/3: OMA=290.3,NOMA_OPT=292.6
GENERAL/0.117-181/0: OMA=285.0,NOMA_OPT=283.6
GENERAL/0.117-181/2: OMA=302.6,NOMA_OPT=307.70000000000005
GENERAL/0.117-181/3: OMA=290.3,NOMA_OPT=292.6
GENERAL/0.117-182/0: OMA=285.0,NOMA_OPT=283.6
GENERAL/0.117-182/2: OMA=302.6,NOMA_OPT=307.70000000000005
GENERAL/0.117-182/3: OMA=290.3,NOMA_OPT=292.6
GENERAL/0.117-183/0: OMA=285.0,NOMA_OPT=283.6
GENERAL/0.117-183/2: OMA=302.6,NOMA_OPT=307.70000000000005
GENERAL/0.117-183/3: OMA=290.3,NOMA_OPT=292.6
GENERAL

GENERAL/0.176-153/3: OMA=310.1,NOMA_OPT=317.40000000000003
GENERAL/0.176-153/4: OMA=298.8,NOMA_OPT=300.7
GENERAL/0.176-153/5: OMA=296.0,NOMA_OPT=-1
GENERAL/0.176-154/0: OMA=295.6,NOMA_OPT=293.8
GENERAL/0.176-154/2: OMA=322.40000000000003,NOMA_OPT=-1
GENERAL/0.176-154/3: OMA=310.1,NOMA_OPT=317.40000000000003
GENERAL/0.176-154/4: OMA=298.8,NOMA_OPT=300.7
GENERAL/0.176-154/5: OMA=296.0,NOMA_OPT=-1
GENERAL/0.176-155/0: OMA=295.6,NOMA_OPT=294.3
GENERAL/0.176-155/2: OMA=322.40000000000003,NOMA_OPT=-1
GENERAL/0.176-155/3: OMA=310.1,NOMA_OPT=317.40000000000003
GENERAL/0.176-155/4: OMA=298.90000000000003,NOMA_OPT=300.90000000000003
GENERAL/0.176-155/5: OMA=296.0,NOMA_OPT=-1
GENERAL/0.176-156/0: OMA=295.7,NOMA_OPT=294.3
GENERAL/0.176-156/2: OMA=322.90000000000003,NOMA_OPT=-1
GENERAL/0.176-156/3: OMA=310.20000000000005,NOMA_OPT=317.40000000000003
GENERAL/0.176-156/4: OMA=298.90000000000003,NOMA_OPT=300.90000000000003
GENERAL/0.176-156/5: OMA=296.0,NOMA_OPT=-1
GENERAL/0.176-157/0: OMA=295.7,NOMA_O

GENERAL/1.189-160/0: OMA=308.90000000000003,NOMA_OPT=307.70000000000005
GENERAL/1.189-160/1: OMA=313.20000000000005,NOMA_OPT=318.0
GENERAL/1.189-161/0: OMA=308.90000000000003,NOMA_OPT=307.70000000000005
GENERAL/1.189-161/1: OMA=313.20000000000005,NOMA_OPT=318.0
GENERAL/1.189-162/0: OMA=308.90000000000003,NOMA_OPT=307.70000000000005
GENERAL/1.189-162/1: OMA=313.20000000000005,NOMA_OPT=318.0
GENERAL/1.189-163/0: OMA=308.90000000000003,NOMA_OPT=307.70000000000005
GENERAL/1.189-163/1: OMA=313.20000000000005,NOMA_OPT=318.0
GENERAL/0.203-6/0: OMA=308.90000000000003,NOMA_OPT=307.70000000000005
GENERAL/0.203-6/1: OMA=313.3,NOMA_OPT=318.0
GENERAL/0.203-7/0: OMA=309.0,NOMA_OPT=307.70000000000005
GENERAL/0.203-7/1: OMA=313.3,NOMA_OPT=318.5
GENERAL/0.203-8/0: OMA=309.0,NOMA_OPT=307.70000000000005
GENERAL/0.203-8/1: OMA=313.3,NOMA_OPT=318.5
GENERAL/0.203-9/0: OMA=309.0,NOMA_OPT=307.70000000000005
GENERAL/0.203-9/1: OMA=313.40000000000003,NOMA_OPT=318.5
GENERAL/0.203-10/0: OMA=309.0,NOMA_OPT=307.700

In [97]:
 def BaseStationThroughPutReport(self):
        sg_stats = {}
        bs_bits = [0 for _ in GV.NET_STATION_CONTROLLER]
        GV.STATISTIC.Log("=====BaseStationThroughPutReport=====")
        for sg in SocialGroup:
            for record in self.sg_header[sg].values():
                for bs_serial,bs_config in enumerate(GV.NET_STATION_CONTROLLER):
                    if(record.time_bs_serv[bs_serial] > 0):
                        bs_bits[bs_serial] += record.bits
        for bs_type in BaseStationType:
            bs_type_num = 0
            bs_type_bits = 0
            for bs_serial in [bs_serial for bs_serial,bs_config in enumerate(GV.NET_STATION_CONTROLLER) if bs_config["type"] == bs_type]:
                bs_type_bits += bs_bits[bs_serial]
                bs_type_num += 1
            bs_type_through_put_avg = (
                (bs_type_bits / max(bs_type_num, 1)) /
                SUMO_SIM_SECONDS
            )
            print("====={}=====".format(bs_type.name))
            print("Through Put:{:.2f}/s".format(bs_type_through_put_avg))
            sg_stats[bs_type] = bs_type_through_put_avg
        return sg_stats

In [98]:
BaseStationThroughPutReport(statistic)

=====BaseStationThroughPutReport=====
=====UMI=====
Through Put:456557.22/s
=====UMA=====
Through Put:697338.22/s


{<BaseStationType.UMI: 0>: 456557.22222222225,
 <BaseStationType.UMA: 1>: 697338.2222222222}